# Bible Word Analyser

## How to use

Enter a strongs number to the text box, and run all cells

In [ ]:
strongs_number = input("Enter a Strong's number: ")
print ("You have selected: " + strongs_number)

Enter a Strong's number: 334
You have selected: 334


# Reading Data

## Raw Dataset Info
 * https://github.com/openscriptures/morphhb/tree/master - hebrew bible with strongs numbers - lemma is the strongs number. XML format, but can be converted to json

 * https://github.com/openscriptures/HebrewLexicon/tree/master - use this to see what hebrew word is represented by strongs number - XML concordance

* https://github.com/openscriptures/GreekResources/tree/master - json with lexicon containing strongs for the Septuagint. Useful, but maybe not needed at this point

* https://github.com/openscriptures/strongs/blob/master/greek/strongs-greek-dictionary.js - strongs dictionary for hebrew, and importantly, greek

* OpenGNT_BASE_TEXT.zip from https://github.com/eliranwong/OpenGNT/tree/master - stores the greek nt and strongs numbers corresponding to each word

## Generate dictionary of Hebrew words and strongs numbers

In [6]:
"""Download the files"""
from urllib.request import urlretrieve

# The names of each hebrew book to download
book_names = [
    "1Chr",
    "1Kgs",
    "1Sam",
    "2Chr",
    "2Kgs",
    "2Sam",
    "Amos",
    "Dan",
    "Deut",
    "Eccl",
    "Esth",
    "Exod",
    "Ezek",
    "Ezra",
    "Gen"
]
url = "https://raw.githubusercontent.com/openscriptures/morphhb/master/wlc/Hos.xml"
filename = "hebrew/file.xml"
urlretrieve(url, filename)

('hebrew/file.xml', <http.client.HTTPMessage at 0x784299f27880>)